In [1]:
import numpy as np
import pandas as pd
import pyspark 
from pyspark import *
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.sql import SparkSession

In [2]:
df = sqlContext.sql("SELECT * FROM attrition_csv")

In [3]:
df.columns

Out[ 303 ]: 
['Index',
 'Age',
 'Attrition',
 'BusinessTravel',
 'DailyRate',
 'Department',
 'DistanceFromHome',
 'Education',
 'EducationField',
 'EmployeeCount',
 'EmployeeNumber',
 'EnvironmentSatisfaction',
 'Gender',
 'HourlyRate',
 'JobInvolvement',
 'JobLevel',
 'JobRole',
 'JobSatisfaction',
 'MaritalStatus',
 'MonthlyIncome',
 'MonthlyRate',
 'NumCompaniesWorked',
 'Over18',
 'OverTime',
 'PercentSalaryHike',
 'PerformanceRating',
 'RelationshipSatisfaction',
 'StandardHours',
 'StockOptionLevel',
 'TotalWorkingYears',
 'TrainingTimesLastYear',
 'WorkLifeBalance',
 'YearsAtCompany',
 'YearsInCurrentRole',
 'YearsSinceLastPromotion',
 'YearsWithCurrManager']

In [4]:
indexer1 = StringIndexer(inputCol = 'MaritalStatus' , outputCol = 'marital_status')
df = indexer1.fit(df).transform(df)

In [5]:
indexer2 = StringIndexer(inputCol = 'OverTime' , outputCol = 'over_time')
df = indexer2.fit(df).transform(df)

In [6]:
df.printSchema()


root
-- Index: integer (nullable = true)
-- Age: integer (nullable = true)
-- Attrition: string (nullable = true)
-- BusinessTravel: integer (nullable = true)
-- DailyRate: integer (nullable = true)
-- Department: string (nullable = true)
-- DistanceFromHome: integer (nullable = true)
-- Education: integer (nullable = true)
-- EducationField: string (nullable = true)
-- EmployeeCount: integer (nullable = true)
-- EmployeeNumber: integer (nullable = true)
-- EnvironmentSatisfaction: integer (nullable = true)
-- Gender: integer (nullable = true)
-- HourlyRate: integer (nullable = true)
-- JobInvolvement: integer (nullable = true)
-- JobLevel: integer (nullable = true)
-- JobRole: string (nullable = true)
-- JobSatisfaction: integer (nullable = true)
-- MaritalStatus: string (nullable = true)
-- MonthlyIncome: integer (nullable = true)
-- MonthlyRate: integer (nullable = true)
-- NumCompaniesWorked: integer (nullable = true)
-- Over18: string (nullable = true)
-- OverTime: string (nullable = true)
-- PercentSalaryHike: integer (nullable = true)
-- PerformanceRating: integer (nullable = true)
-- RelationshipSatisfaction: integer (nullable = true)
-- StandardHours: integer (nullable = true)
-- StockOptionLevel: integer (nullable = true)
-- TotalWorkingYears: integer (nullable = true)
-- TrainingTimesLastYear: integer (nullable = true)
-- WorkLifeBalance: integer (nullable = true)
-- YearsAtCompany: integer (nullable = true)
-- YearsInCurrentRole: integer (nullable = true)
-- YearsSinceLastPromotion: integer (nullable = true)
-- YearsWithCurrManager: integer (nullable = true)
-- marital_status: double (nullable = false)
-- over_time: double (nullable = false)

In [7]:
vector_assembler = VectorAssembler(inputCols = ['Age','BusinessTravel','DailyRate','DistanceFromHome','EnvironmentSatisfaction','Gender','HourlyRate','JobInvolvement','JobLevel','JobSatisfaction','marital_status','MonthlyIncome','NumCompaniesWorked','over_time','PercentSalaryHike','PerformanceRating','StockOptionLevel','TotalWorkingYears','WorkLifeBalance','YearsAtCompany','YearsInCurrentRole','YearsSinceLastPromotion','YearsWithCurrManager'],outputCol = "features")

In [8]:
vectorised_df = vector_assembler.transform(df)

In [9]:
str_indexer = StringIndexer(inputCol = "Attrition" , outputCol = 'label')

In [10]:
indexed_vectorised_df = str_indexer.fit(vectorised_df).transform(vectorised_df)

In [11]:
df = indexed_vectorised_df

In [12]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [13]:
splits = df.randomSplit([0.7,0.3],seed = 1)

In [14]:
training_df = splits[0]
test_df = splits[1]

In [15]:
print(training_df.count())
print(test_df.count())

1025
445

In [16]:
dt_definition  = DecisionTreeClassifier(labelCol = 'label',featuresCol = 'features')


In [17]:
dt_model = dt_definition.fit(training_df)
dt_df = dt_model.transform(test_df)




In [18]:
dt_evaluator = MulticlassClassificationEvaluator(labelCol = 'label',predictionCol = 'prediction',metricName = 'accuracy')
dt_accuracy = dt_evaluator.evaluate(dt_df)
dt_accuracy

Out[ 318 ]: 0.8584269662921349

In [19]:
from pyspark.ml.feature import ChiSqSelector
from pyspark.ml.linalg import Vectors



In [20]:
selector = ChiSqSelector(numTopFeatures=4, featuresCol="features",
                         outputCol="selectedFeatures", labelCol="label")
chi_df = selector.fit(indexed_vectorised_df).transform(indexed_vectorised_df)

splits =chi_df.randomSplit([0.61,0.39],seed = 5)
training_df = splits[0]
test_df = splits[1]
dt_definition  = DecisionTreeClassifier(labelCol = 'label',featuresCol = 'selectedFeatures')
dt_model = dt_definition.fit(training_df)
dt_df = dt_model.transform(test_df)

dt_evaluator = MulticlassClassificationEvaluator(labelCol = 'label',predictionCol = 'prediction',metricName = 'accuracy')
dt_accuracy = dt_evaluator.evaluate(dt_df)
dt_accuracy


Out[ 330 ]: 0.8686514886164624

In [21]:
# try 2 

In [22]:
df = sqlContext.sql("SELECT * FROM attrition_csv")

In [23]:
vector_assembler = VectorAssembler(inputCols = ['Age','DailyRate','DistanceFromHome','Education','EmployeeCount','EnvironmentSatisfaction','HourlyRate','JobInvolvement','JobLevel','JobSatisfaction','MonthlyIncome','NumCompaniesWorked','PercentSalaryHike','PerformanceRating','StockOptionLevel','TotalWorkingYears','WorkLifeBalance','YearsAtCompany','YearsInCurrentRole','YearsSinceLastPromotion','YearsWithCurrManager'],outputCol = "features")

In [24]:

vectorised_df = vector_assembler.transform(df)
indexed_vectorised_df = str_indexer.fit(vectorised_df).transform(vectorised_df)
df = indexed_vectorised_df



In [25]:
splits = df.randomSplit([0.6,0.4],seed = 1)
training_df = splits[0]
test_df = splits[1]

In [26]:
dt_definition  = DecisionTreeClassifier(labelCol = 'label',featuresCol = 'features')
dt_model = dt_definition.fit(training_df)
dt_df = dt_model.transform(test_df)


In [27]:
dt_evaluator = MulticlassClassificationEvaluator(labelCol = 'label',predictionCol = 'prediction',metricName = 'accuracy')
dt_accuracy = dt_evaluator.evaluate(dt_df)
dt_accuracy

Out[ 347 ]: 0.828719723183391

In [28]:
selector = ChiSqSelector(numTopFeatures=10, featuresCol="features",
                         outputCol="selectedFeatures", labelCol="label")
chi_df = selector.fit(indexed_vectorised_df).transform(indexed_vectorised_df)

splits =chi_df.randomSplit([0.9,0.1],seed = 4)
training_df = splits[0]
test_df = splits[1]
dt_definition  = DecisionTreeClassifier(labelCol = 'label',featuresCol = 'selectedFeatures')
dt_model = dt_definition.fit(training_df)
dt_df = dt_model.transform(test_df)

dt_evaluator = MulticlassClassificationEvaluator(labelCol = 'label',predictionCol = 'prediction',metricName = 'accuracy')
dt_accuracy = dt_evaluator.evaluate(dt_df)
dt_accuracy

Out[ 375 ]: 0.9178082191780822